In [4]:
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split

In [5]:
ws = Workspace.from_config()

In [6]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automl-capstone'
project_folder = './automl'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-capstone,endofproj,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
ws.datasets

{'Parkisons-Data': DatasetRegistration(id='3fc264b3-7442-4ee7-98b6-33f755771995', name='Parkisons-Data', version=1, description=' Parkinsons Disease Data Set', tags={})}

In [8]:
parkinsons_ds = Dataset.get_by_name(workspace=ws, name='Parkisons-Data')

In [10]:
parkinsons_df = parkinsons_ds.to_pandas_dataframe()

In [11]:
parkinsons_df.head(5)

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [12]:
parkinsons_df.columns

Index(['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)',
       'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP',
       'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5',
       'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA',
       'spread1', 'spread2', 'D2', 'PPE'],
      dtype='object')

In [13]:
automl_settings = {
    "experiment_timeout_minutes": 15,
    "iteration_timeout_minutes":2,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "enable_early_stopping":True,
    "featurization":'auto'
}

In [14]:
automl_config = AutoMLConfig(
                             task = "classification",
                             training_data=parkinsons_df,
                             label_column_name="status",   
                             path = project_folder,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [15]:
run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration


2022-04-11:14:46:23,157 INFO     [modeling_bert.py:226] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2022-04-11:14:46:23,163 INFO     [modeling_xlnet.py:339] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-capstone,AutoML_5d84289e-7b7a-401f-ad36-71dcf37002e0,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|10                            |
+------------------------------+

********************************************************************************************

TYPE:         Class balancing de

2022-04-11:15:05:13,512 INFO     [explanation_client.py:334] Using default datastore for uploads


Current status: EngineeredFeatureExplanations. Computation of engineered features completed
Current status: RawFeaturesExplanations. Computation of raw features started
Current status: RawFeaturesExplanations. Computation of raw features completed
Current status: BestRunExplainModel. Best run model explanations completed
********************************************************************************************


In [20]:
from azureml.widgets import RunDetails
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl-capstone,AutoML_5d84289e-7b7a-401f-ad36-71dcf37002e0,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|10                            |
+------------------------------+

********************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were dete

{'runId': 'AutoML_5d84289e-7b7a-401f-ad36-71dcf37002e0',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2022-04-11T14:46:27.120393Z',
 'endTimeUtc': '2022-04-11T15:03:03.013975Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 15 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-capstone","subscription_id":"d6ebc258-732e-4a76-8d46-be5af7f5737d","resource_group":"tsar","workspace_name":"endofproj","region":"eastus2","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_

In [23]:
best_run, fitted_model = run.get_output()
print(best_model)

Run(Experiment: automl-capstone,
Id: AutoML_5d84289e-7b7a-401f-ad36-71dcf37002e0_29,
Type: None,
Status: Completed)


In [ ]:
print(fitted_model)
#code line above and below displays parameters of the AutoML model, choose any** 
#fitted_model.steps[1][1].estimators

In [24]:
model_name = best_run.properties['model_name']

script_file_name = 'inference/score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')

2022-04-11:15:18:08,157 INFO     [explanation_client.py:334] Using default datastore for uploads
2022-04-11:15:18:15,620 INFO     [explanation_client.py:334] Using default datastore for uploads
2022-04-11:15:18:23,112 INFO     [explanation_client.py:334] Using default datastore for uploads
2022-04-11:15:18:30,751 INFO     [explanation_client.py:334] Using default datastore for uploads
2022-04-11:15:18:38,617 INFO     [explanation_client.py:334] Using default datastore for uploads


In [25]:

model = run.register_model(model_name = model_name, description = 'AutoML Parkisons model', tags = None)

print("Model ID", run.model_id)

Model ID AutoML5d84289e729


2022-04-11:15:19:27,862 INFO     [explanation_client.py:334] Using default datastore for uploads
2022-04-11:15:19:36,423 INFO     [explanation_client.py:334] Using default datastore for uploads


In [27]:

from azureml.core.environment import Environment
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

2022-04-11:15:20:31,166 INFO     [explanation_client.py:334] Using default datastore for uploads


In [39]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name, environment=myenv)
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               description = 'Parkinsons Disease Classification Service')

aci_service_name = 'parkinsons-service'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

parkinsons-service
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-11 16:40:37+00:00 Creating Container Registry if not exists.
2022-04-11 16:40:37+00:00 Registering the environment.
2022-04-11 16:40:37+00:00 Use the existing image.
2022-04-11 16:40:38+00:00 Generating deployment configuration.
2022-04-11 16:40:39+00:00 Submitting deployment to compute.
2022-04-11 16:40:42+00:00 Checking the status of deployment parkinsons-service.

2022-04-11:16:41:08,455 INFO     [explanation_client.py:334] Using default datastore for uploads
2022-04-11:16:42:09,215 INFO     [explanation_client.py:334] Using default datastore for uploads
2022-04-11:16:43:09,936 INFO     [explanation_client.py:334] Using default datastore for uploads


.
2022-04-11 16:43:43+00:00 Checking the status of inference endpoint parkinsons-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [41]:
# URL for the web service, should be similar to:

scoring_uri = 'http://f15c618e-6f83-4b45-b033-a7beda3fe9c6.eastus2.azurecontainer.io/score'

data = {
  "data": [
    {
      "D2": 2.89,
      "DFA": 0.622,
      "HNR": 23.0,
      "Jitter:DDP": 0.00421,
      "MDVP:APQ": 0.0455,
      "MDVP:Fhi(Hz)": 200.9,
      "MDVP:Flo(Hz)": 93.98,
      "MDVP:Fo(Hz)": 156.6,
      "MDVP:Jitter(%)": 0.0089,
      "MDVP:Jitter(Abs)": 0.00006,
      "MDVP:PPQ": 0.00292,
      "MDVP:RAP": 0.0022,
      "MDVP:Shimmer": 0.03327,
      "MDVP:Shimmer(dB)": 0.482,
      "NHR": 0.0341,
      "PPE": 0.0168,
      "RPDE": 0.622,
      "Shimmer:APQ3": 0.0288,
      "Shimmer:APQ5": 0.0045,
      "Shimmer:DDA": 0.05,
      "name": "example_value",
      "spread1": -7.082,
      "spread2": 0.336
    },
  ],
  "method": "predict"

    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}


# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [1]}


2022-04-11:16:49:13,310 INFO     [explanation_client.py:334] Using default datastore for uploads


In [42]:
print(resp.status_code)
print(resp.elapsed)
print(resp.json())

200
0:00:00.247466
{"result": [1]}


In [45]:
## replace <service_name> with the name of the web service
aci_service.update(enable_app_insights=True)

In [46]:
aci_service.delete()

2022-04-11:17:13:26,645 INFO     [explanation_client.py:334] Using default datastore for uploads
